# Installing Dependencies

In [1]:
!pip install pandas
!pip install numpy

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd;
import numpy as np;
import random;

# Loading the Data

In [3]:
df_original = pd.read_csv('dataset/participant_data.csv');
column_names = df_original.columns;
for col in column_names: print("Feature: " + col);
df_original

Feature: _time
Feature: agentID
Feature: Altitude
Feature: Latitude
Feature: Longitude
Feature: http_result
Feature: icmp_result
Feature: tcp_result
Feature: udp_result


,_time,agentID,Altitude,Latitude,Longitude,http_result,icmp_result,tcp_result,udp_result
0,2024-09-20 17:19:05+00:00,752813491.0,20.292809,45.496175,-73.561853,29.991,49.161,30.106,58.090
1,2024-09-20 17:19:03+00:00,752813491.0,20.292809,45.496175,-73.561853,29.955,59.476,30.031,69.706
2,2024-09-20 17:19:01+00:00,752813491.0,20.292809,45.496175,-73.561853,33.209,31.992,29.947,42.514
3,2024-09-20 17:18:59+00:00,752813491.0,20.292809,45.496175,-73.561853,30.057,64.112,30.036,74.137
4,2024-09-20 17:18:57+00:00,752813491.0,20.292809,45.496175,-73.561853,39.228,43.609,30.135,53.984
...,...,...,...,...,...,...,...,...,...
2593,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2594,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2595,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2596,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
origData = df_original.to_numpy();
randSample = origData[random.randint(0,len(origData)-1)]; # for inspection

print("Type of Array with original data: " + str(type(origData)));
print("Type of a record in the original data: " + str(type(randSample)));
print("Number of records in the original dataset: " + str(len(origData)));
print("Number of features in a record: " + str(randSample.size));
print("Dimensionality of a record: " + str(randSample.ndim));
print("Dimensionality of the entire data array: " + str(origData.ndim));
print("Shape of the Array: " + str(origData.shape));
print("Shape of a record: " + str(randSample.shape));

Type of Array with original data: <class 'numpy.ndarray'>
Type of a record in the original data: <class 'numpy.ndarray'>
Number of records in the original dataset: 2598
Number of features in a record: 9
Dimensionality of a record: 1
Dimensionality of the entire data array: 2
Shape of the Array: (2598, 9)
Shape of a record: (9,)


We saw NaN values in the table. That is because of empty cells in the table. To prove our intuition, let's inspect whether there are NaN in specific columns.

In [6]:
df_original.isna().any()

_time          True
agentID        True
Altitude       True
Latitude       True
Longitude      True
http_result    True
icmp_result    True
tcp_result     True
udp_result     True
dtype: bool

# Data Cleansing

Delete empty rows

In [7]:
# delete empty rows which has empty timestamp (we assume this is the primary key of a record)
df_noEmptyRows = df_original.dropna(subset=['_time'])

Check if there're empty rows now

In [8]:
df_noEmptyRows['_time'].isnull()

0       False
1       False
2       False
3       False
4       False
        ...  
1930    False
1931    False
1932    False
1933    False
1934    False
Name: _time, Length: 1935, dtype: bool

Check if there're still NaN / Empty cells

In [11]:
df_noEmptyRows.isna().any()

_time          False
agentID        False
Altitude        True
Latitude       False
Longitude      False
http_result     True
icmp_result     True
tcp_result      True
udp_result      True
dtype: bool

## Approach 1: Replace NaN values (from empty numeric cells) with the mean of the column

We need to replace all the NaN values with the mean of the data.

In [12]:
# obtain the mean from numeric columns only
means = df_noEmptyRows.mean(numeric_only=True);
means

agentID        7.528135e+08
Altitude       2.283825e+01
Latitude       4.549289e+01
Longitude     -7.356152e+01
http_result    4.478491e+01
icmp_result    1.677274e+03
tcp_result     4.244834e+01
udp_result     1.278443e+03
dtype: float64

In [13]:
df_cleaned_mean = df_noEmptyRows[1:].fillna(means)
df_cleaned_mean

,_time,agentID,Altitude,Latitude,Longitude,http_result,icmp_result,tcp_result,udp_result
1,2024-09-20 17:19:03+00:00,752813491.0,20.292809,45.496175,-73.561853,29.955,59.476,30.031,69.706
2,2024-09-20 17:19:01+00:00,752813491.0,20.292809,45.496175,-73.561853,33.209,31.992,29.947,42.514
3,2024-09-20 17:18:59+00:00,752813491.0,20.292809,45.496175,-73.561853,30.057,64.112,30.036,74.137
4,2024-09-20 17:18:57+00:00,752813491.0,20.292809,45.496175,-73.561853,39.228,43.609,30.135,53.984
5,2024-09-20 17:18:55+00:00,752813491.0,20.292809,45.496175,-73.561853,34.133,35.945,30.996,44.054
...,...,...,...,...,...,...,...,...,...
1930,2024-09-16 17:13:27+00:00,752813491.0,18.823519,45.496026,-73.561997,12.473,11.588,12.961,14.024
1931,2024-09-16 17:13:25+00:00,752813491.0,18.823519,45.496026,-73.561997,22.238,12.144,12.141,20.624
1932,2024-09-16 17:13:23+00:00,752813491.0,18.823519,45.496026,-73.561997,14.019,17.161,14.217,24.523
1933,2024-09-16 17:13:21+00:00,752813491.0,18.823519,45.496026,-73.561997,11.645,12.584,13.018,17.155


Check if there're still NaN / Empty cells

In [14]:
df_cleaned_mean.isna().any()

_time          False
agentID        False
Altitude       False
Latitude       False
Longitude      False
http_result    False
icmp_result    False
tcp_result     False
udp_result     False
dtype: bool

Make sure the data fields remain unchanged

In [15]:
cleanedData = df_cleaned_mean.to_numpy();
randSample = cleanedData[random.randint(0,len(cleanedData)-1)]; # for inspection

print("Type of Array with original data: " + str(type(cleanedData)));
print("Type of a record in the original data: " + str(type(randSample)));
print("Number of records in the original dataset: " + str(len(cleanedData)));
print("Number of features in a record: " + str(randSample.size));
print("Dimensionality of a record: " + str(randSample.ndim));
print("Dimensionality of the entire data array: " + str(cleanedData.ndim));
print("Shape of the Array: " + str(cleanedData.shape));
print("Shape of a record: " + str(randSample.shape));

Type of Array with original data: <class 'numpy.ndarray'>
Type of a record in the original data: <class 'numpy.ndarray'>
Number of records in the original dataset: 1934
Number of features in a record: 9
Dimensionality of a record: 1
Dimensionality of the entire data array: 2
Shape of the Array: (1934, 9)
Shape of a record: (9,)


Exporting the cleaned data to CSV

In [16]:
df_cleaned_mean.to_csv('dataset/participant_data_cleaned_mean.csv');

## Approach 2: Replace NaN values (from empty numeric cells) with the median of the column

We need to replace all the NaN values with the mean of the data.

In [17]:
# obtain the mean from numeric columns only
medians = df_noEmptyRows.median(numeric_only=True);
medians

agentID        7.528135e+08
Altitude       1.973038e+01
Latitude       4.549392e+01
Longitude     -7.356011e+01
http_result    3.087150e+01
icmp_result    4.773400e+01
tcp_result     3.008900e+01
udp_result     5.215500e+01
dtype: float64

In [18]:
df_cleaned_median = df_noEmptyRows[1:].fillna(medians)
df_cleaned_median

,_time,agentID,Altitude,Latitude,Longitude,http_result,icmp_result,tcp_result,udp_result
1,2024-09-20 17:19:03+00:00,752813491.0,20.292809,45.496175,-73.561853,29.955,59.476,30.031,69.706
2,2024-09-20 17:19:01+00:00,752813491.0,20.292809,45.496175,-73.561853,33.209,31.992,29.947,42.514
3,2024-09-20 17:18:59+00:00,752813491.0,20.292809,45.496175,-73.561853,30.057,64.112,30.036,74.137
4,2024-09-20 17:18:57+00:00,752813491.0,20.292809,45.496175,-73.561853,39.228,43.609,30.135,53.984
5,2024-09-20 17:18:55+00:00,752813491.0,20.292809,45.496175,-73.561853,34.133,35.945,30.996,44.054
...,...,...,...,...,...,...,...,...,...
1930,2024-09-16 17:13:27+00:00,752813491.0,18.823519,45.496026,-73.561997,12.473,11.588,12.961,14.024
1931,2024-09-16 17:13:25+00:00,752813491.0,18.823519,45.496026,-73.561997,22.238,12.144,12.141,20.624
1932,2024-09-16 17:13:23+00:00,752813491.0,18.823519,45.496026,-73.561997,14.019,17.161,14.217,24.523
1933,2024-09-16 17:13:21+00:00,752813491.0,18.823519,45.496026,-73.561997,11.645,12.584,13.018,17.155


Check if there're still NaN / Empty cells

In [19]:
df_cleaned_median.isna().any()

_time          False
agentID        False
Altitude       False
Latitude       False
Longitude      False
http_result    False
icmp_result    False
tcp_result     False
udp_result     False
dtype: bool

Make sure the data fields remain unchanged

In [21]:
cleanedData = df_cleaned_median.to_numpy();
randSample = cleanedData[random.randint(0,len(cleanedData)-1)]; # for inspection

print("Type of Array with original data: " + str(type(cleanedData)));
print("Type of a record in the original data: " + str(type(randSample)));
print("Number of records in the original dataset: " + str(len(cleanedData)));
print("Number of features in a record: " + str(randSample.size));
print("Dimensionality of a record: " + str(randSample.ndim));
print("Dimensionality of the entire data array: " + str(cleanedData.ndim));
print("Shape of the Array: " + str(cleanedData.shape));
print("Shape of a record: " + str(randSample.shape));

Type of Array with original data: <class 'numpy.ndarray'>
Type of a record in the original data: <class 'numpy.ndarray'>
Number of records in the original dataset: 1934
Number of features in a record: 9
Dimensionality of a record: 1
Dimensionality of the entire data array: 2
Shape of the Array: (1934, 9)
Shape of a record: (9,)


Exporting the cleaned data to CSV

In [22]:
df_cleaned_median.to_csv('dataset/participant_data_cleaned_median.csv');